In [123]:
# Functional -- 4/11/2024
import requests
from bs4 import BeautifulSoup
import re

# The base URL for UFC results
base_url = "https://www.ufc.com"
results_url = base_url + "/results?language_content_entity=en"

def extract_url_text_to_compare_previous_link(url):
    """
    Extracts a unique identifier from a UFC event URL, focusing on the part between 'ufc' and 'vs'.
    This is used to track the 'event' portion of URLs and to help detect redundant entries based on event details.
    If the URL does not match the expected pattern, the function returns None and prints a message.
    """
    match = re.search(r'ufc-(.*?)-vs', url)
    if match:
        return match.group(1)
    
    print("No unique identifier found")
    return None


def extract_unique_identifier(url):
    """
    Extracts a broader unique identifier from a UFC event URL, capturing the part after '/news/'.
    This helps to identify event URLs with varying structures, improving data coverage and preventing duplicate entries.
    If the URL does not match the expected format, the function returns None and prints a message.
    """
    match = re.search(r'/news/(.*)', url)
    if match:
        return match.group(1)
    
    print("No unique identifier found")
    return None


def find_results_links(url):
    """
    Navigates through the UFC results page, gathering links to individual fight result pages.
    Filters out unnecessary links (e.g., weigh-ins or bonus announcements) and checks for redundancy by
    comparing the unique identifiers from each link. Uses a 'Load more' button if available to paginate.
    """
    results_links = []  # Initialize a list to store collected results links
    last_fight_info = None  # Track last event identifier to prevent redundant entries
    
    while url:
        print("Processing:", url)  # Debug: show the current page URL being processed
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all relevant event result links on the page
        for link in soup.find_all("a", class_="c-card--grid-card-trending"):
            link_text = link.text.lower()
            link_href = link.get('href')
            # Filter by links containing 'results' but excluding weigh-ins, bonuses, etc.
            if "results" in link_text and "weigh" not in link_text and "bonus" not in link_text and "episode" not in link_text and "season" not in link_text:
                # Extract event identifier to prevent duplicate links for the same event
                current_fight_info = extract_url_text_to_compare_previous_link(link_href)
                if current_fight_info != last_fight_info:
                    full_link = base_url + link_href
                    results_links.append(full_link)
                    last_fight_info = current_fight_info  # Update last identifier for comparison
        
        # Check for 'Load more' button to proceed to next page of results
        load_more_button = soup.find("a", title="Load more items")
        if load_more_button:
            next_page = load_more_button.get('href')
            # Adjust for relative URL if present, otherwise proceed to the next absolute URL
            if next_page.startswith("?"):
                url = base_url + "/results" + next_page
            else:
                url = next_page
        else:
            url = None  # Stop if no more pages are available
            
    return results_links


def find_event_scorecard_links(results_links):
    """
    Processes each result link to identify scorecard links associated with each event.
    Checks for duplicate event identifiers and fetches official scorecard links if available.
    For links without explicit scorecards, it opens the link to search for 'official' scorecard links within the page.
    """
    event_scorecard_links = []  # List to store the final set of scorecard links
    last_identifier = None  # Track last identifier to avoid duplicate scorecard entries
    
    for url in results_links:
        print("Processing:", url)  # Debug: show the URL currently being processed
        # Extract unique identifier to detect if the link represents a duplicate event
        current_identifier = extract_unique_identifier(url)
        if current_identifier != last_identifier:
            # If the URL already contains 'official scorecard', add it directly to the list
            if "official-judges-scorecard" in url or "official-scorecard" in url:
                print("result_link same as scorecard_link:", url)
                event_scorecard_links.append(url)
                last_identifier = current_identifier
            else:
                print("Visiting result_link:", url)  # Debug: opening link to look for scorecard
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')

                # Find links containing 'official' and 'scorecard' within the page
                scorecard_link_element = soup.find("a", href=lambda href: href and "official" in href and "scorecard" in href)
                if scorecard_link_element:
                    scorecard_link = scorecard_link_element.get('href')
                    # Adjust for relative URLs if necessary
                    formatted_scorecard_link = scorecard_link if scorecard_link.startswith("http") else base_url + scorecard_link
                    
                    # Re-extract identifier for the updated URL structure, if different
                    new_identifier = extract_unique_identifier(formatted_scorecard_link)
                    if new_identifier != last_identifier:
                        event_scorecard_links.append(formatted_scorecard_link)
                        last_identifier = new_identifier
    
    return event_scorecard_links


def remove_redundant_links(event_scorecard_links):
    """
    Removes redundant links by:
    1. Filtering out any links containing the 'search?query=official+scorecards' keyword.
    2. Checking for links that are substrings of others, indicating redundancy.
    """
    removed_event_scorecard_links = []  # List of links removed
    filtered_links = []  # List of links kept after filtering
    links_to_remove = set()  # Set for efficient removal

    # Filter out links containing 'search?query=official+scorecards'
    for link in event_scorecard_links:
        if 'search?query=official+scorecards' in link:
            links_to_remove.add(link)
    
    # Check each link against all others to detect redundancies
    for i, link_i in enumerate(event_scorecard_links):
        for j, link_j in enumerate(event_scorecard_links):
            # Avoid self-comparison
            if i == j:
                continue
            # Mark link_j for removal if link_i is contained within it
            if link_i in link_j and link_j not in links_to_remove:
                links_to_remove.add(link_j)
    
    # Separate filtered and removed links based on the removal set
    for link in event_scorecard_links:
        if link in links_to_remove:
            removed_event_scorecard_links.append(link)
        else:
            filtered_links.append(link)
    
    return filtered_links, removed_event_scorecard_links


def add_manual_event_scorecard_links(existing_links, manual_links):
    """
    Adds manually identified scorecard links that may not have been captured in automated scraping.
    Ensures that each link is only added once to avoid redundancy.
    """
    existing_set = set(existing_links)  # Set for quick lookup of existing links
    combined_links = list(existing_links)  # Initialize with existing links
    for link in manual_links:
        # Only add the link if it does not already exist in the set
        if link not in existing_set:
            combined_links.append(link)
            existing_set.add(link)
    return combined_links

# Manual links identified that were missed by the scraping process
manual_event_scorecard_links = [
    "https://www.ufc.com/news/official-judges-scorecards-ufc-fight-night-ankalaev-vs-walker-2",
    "https://www.ufc.com/news/official-scorecards-ufc-fight-night-ortega-vs-rodriguez",
    "https://www.ufc.com/news/official-judges-scorecards-ufc-269-oliveira-vs-poirier-nunes-pena?language_content_entity=en",
    "https://www.ufc.com/news/official-scorecards-ufc-fight-night-sandhagen-vs-dillashaw",
    "https://www.ufc.com/news/ufc-fight-night-brunson-vs-holland-scorecards-gillespie-riddell-winners-ufc-vegas-22?language_content_entity=en",
    "https://www.ufc.com/news/ufc-257-official-scorecards-poirier-mcgregor-2-fight-island",
    "https://www.ufc.com/news/ufc-fight-island-6-official-scorecards-ortega-korean-zombie-results",
    "https://www.ufc.com/news/ufc-vegas-9-official-scorecards-sakai-overeem",
    "https://www.ufc.com/news/ufc-vegas-8-official-scorecards-fight-night-smith-rakic?language_content_entity=en",
    "https://www.ufc.com/news/ufc-vegas-7-official-scorecards-munhoz-edgar",
]

# Start the process by collecting result links
results_links = find_results_links(results_url)
# Find scorecard links from result links
event_scorecard_links = find_event_scorecard_links(results_links)
# Remove redundant links and capture those removed
event_scorecard_links, removed_event_scorecard_links = remove_redundant_links(event_scorecard_links)
# Add manually identified scorecard links to the final list
event_scorecard_links = add_manual_event_scorecard_links(event_scorecard_links, manual_event_scorecard_links)

# Print out all found event scorecard links
print("\nFound event scorecard links:")
for link in event_scorecard_links:
    print(link)

# Print out removed event scorecard links
print("\nRemoved event scorecard links:")
for removed_link in removed_event_scorecard_links:
    print(removed_link)

# Print out manually added event scorecard links
print("\nManually added event scorecard links:")
for manual_link in manual_event_scorecard_links:
    print(manual_link)


Processing: https://www.ufc.com/results?language_content_entity=en
No unique identifier found
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=1
Processing: https://www.ufc.com/results?language_content_entity=en&page=2
No unique identifier found
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=3
Processing: https://www.ufc.com/results?language_content_entity=en&page=4
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=5
Processing: https://www.ufc.com/results?language_content_entity=en&page=6
Processing: https://www.ufc.com/results?language_content_entity=en&page=7
Processing: https://www.ufc.com/results?language_content_entity=en&page=8
Processing: https://www.ufc.com/results?language_content_entity=en&page=9
Processing: https://www.ufc.com/results?language_content_entity=en&page=10
Processing: https://www.ufc.com/results?language_content_

In [ ]:
# Functions properly 4.11.2024
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from urllib.request import urlretrieve
import img2pdf
import re

def parse_folder_name_from_event_scorecard_link(event_scorecard_link):
    """
    Parses a folder name from the event scorecard link by extracting the part after 'news/' and before a query 
    parameter or end of the string. It replaces hyphens with underscores to make it compatible with file system 
    requirements and removes invalid characters.
    """
    match = re.search(r'news/(.*?)(\?|results|$)', event_scorecard_link)
    if match:
        # Replace '-' with '_' for file system compatibility
        folder_name = match.group(1).replace('-', '_').strip()
        # Remove or replace characters that are invalid in file names
        folder_name = re.sub(r'[\\/*?:"<>|]', '', folder_name)
        return folder_name
    else:
        return "No_Folder_Name_Parsed"

def download_images_from_event_scorecard_link_and_create_pdf(event_scorecard_link, event_index):
    """
    Downloads all images from the specified event scorecard link and saves them into a PDF.
    Uses BeautifulSoup to find image tags, retrieves each image, and then compiles them into a PDF file.
    """
    # Parse folder name for saving the images and PDF
    parsed_folder_name = parse_folder_name_from_event_scorecard_link(event_scorecard_link)
    save_dir = f"ufc_images/event_{event_index}_{parsed_folder_name}"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Send request to the event scorecard link
    response = requests.get(event_scorecard_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find all images tagged with "image-style-inline"
    img_tags = soup.find_all('img', class_="image-style-inline")
    # Filter images by checking if they are hosted on "dmxg5wxfqgb4u.cloudfront.net"
    images = [img for img in img_tags if "dmxg5wxfqgb4u.cloudfront.net" in img.get('src', '')]

    image_paths = []  # List to store downloaded image paths

    # Download each image and store it locally
    for img in images:
        img_url = urljoin(event_scorecard_link, img['src'])
        clean_filename = os.path.basename(urlparse(img_url).path)
        filename = os.path.join(save_dir, clean_filename)

        try:
            # Download the image
            urlretrieve(img_url, filename)
            print(f"Downloaded {filename}")
            image_paths.append(filename)
        except Exception as e:
            print(f"Could not download {img_url}. Reason: {e}")

    # If images were downloaded, create a PDF from them
    if image_paths:
        # Construct the PDF filename within the save directory
        pdf_filename = os.path.join(save_dir, f"{event_index}_{parsed_folder_name}.pdf")
        try:
            with open(pdf_filename, "wb") as f:
                # Convert images to PDF and save
                f.write(img2pdf.convert(image_paths))
            print(f"Created PDF: {pdf_filename}")
        except Exception as e:
            print(f"Failed to create PDF: {pdf_filename}. Error: {e}")


        
# Specify the range of events to process
starting_event_number = 1
ending_event_number = 150

# Loop through the specified range of event scorecard links
for index, event_scorecard_link in enumerate(event_scorecard_links[starting_event_number - 1:ending_event_number], start=starting_event_number):
    print(f"Processing images from: {event_scorecard_link}")
    download_images_from_event_scorecard_link_and_create_pdf(event_scorecard_link, index)


In [132]:
# Simple function to determine if any pdf's were not properly compiled and put into the appropriate folders
import os

def find_folders_without_pdfs(parent_directory):
    """
    Finds and returns a list of folder names within the specified parent directory
    that do not contain any PDF files.

    Args:
    - parent_directory (str): The path to the parent directory to search within.

    Returns:
    - list: A list of folder names without any PDF files.
    """
    folders_without_pdfs = []

    # Walk through all directories and subdirectories of the parent directory
    for root, dirs, files in os.walk(parent_directory):
        # Check if there are any PDF files in the current directory
        if not any(file.endswith('.pdf') for file in files):
            # If there are no PDF files, add the folder name to the list
            # The folder name is the last part of the path in 'root'
            folder_name = os.path.basename(root)
            folders_without_pdfs.append(folder_name)

    return folders_without_pdfs

# Specify the path to the parent directory
parent_directory = r"C:\Users\EditZ\ufc_images"

# Find folders without PDFs
folders_without_pdfs = find_folders_without_pdfs(parent_directory)

# Print the list of folders without PDFs
print("Folders without PDF files:")
for folder in folders_without_pdfs:
    print(folder)


Folders without PDF files:
ufc_images
event_123_ufc_fight_night_whittaker_vs_gastelum_official_scorecards_stephens_klose_arlovski_sherman_espn_plus_ufc_vegas_24
event_124_ufc_fight_night_vettori_vs_holland_official_scorecards_till_allen_yusuff_nunes_dern_abc_espn_plus_ufc_vegas_23


In [136]:
# There was an issue with the automated PDF creation within several folders.
# Here's a simple program to do it manually.

import os
import img2pdf

def create_pdf_from_images(folder_path):
    """
    Creates a PDF file from all JPEG images in the specified folder.
    Checks if the folder exists and collects all .jpg and .jpeg files,
    then compiles them into a single PDF.
    """
    if not os.path.isdir(folder_path):
        print(f"The folder {folder_path} does not exist.")
        return

    # Gather all JPEG images in the folder and sort by filename
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.jpeg')]
    image_files.sort()

    # Use a simpler name for the PDF file to avoid potential issues with long paths
    pdf_filename = os.path.join(folder_path, "compiled_images.pdf")

    if image_files:
        # Construct the full paths to each image for PDF creation
        image_paths = [os.path.join(folder_path, f) for f in image_files]
        try:
            # Create PDF from the images
            with open(pdf_filename, "wb") as f:
                f.write(img2pdf.convert(image_paths))
            print(f"PDF created successfully: {pdf_filename}")
        except Exception as e:
            print(f"Failed to create PDF from images in {folder_path}. Error: {e}")
    else:
        print(f"No JPEG images found in {folder_path}.")

# Define folders to process, each containing images to be compiled into a PDF
folders_to_process = [
    r"C:\Users\EditZ\ufc_images\event_123_ufc_fight_night_whittaker_vs_gastelum_official_scorecards_stephens_klose_arlovski_sherman_espn_plus_ufc_vegas_24",
    r"C:\Users\EditZ\ufc_images\event_124_ufc_fight_night_vettori_vs_holland_official_scorecards_till_allen_yusuff_nunes_dern_abc_espn_plus_ufc_vegas_23"
]

# Process each folder, creating a PDF for the images it contains
for folder in folders_to_process:
    create_pdf_from_images(folder)


PDF created successfully: C:\Users\EditZ\ufc_images\event_123_ufc_fight_night_whittaker_vs_gastelum_official_scorecards_stephens_klose_arlovski_sherman_espn_plus_ufc_vegas_24\compiled_images.pdf
PDF created successfully: C:\Users\EditZ\ufc_images\event_124_ufc_fight_night_vettori_vs_holland_official_scorecards_till_allen_yusuff_nunes_dern_abc_espn_plus_ufc_vegas_23\compiled_images.pdf
